### Mount G-drive

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
cd /

In [ ]:
!mkdir -p /drive
!google-drive-ocamlfuse drive

In [ ]:
cd /content/

In [ ]:
# Copy files into /content/ folder and unzip it
!cp /drive/CS235/cs235proj/*.zip /content/
!unzip breaking98_png.zip
!unzip locking98_png.zip
!unzip popping99_png.zip
!unzip shuffle107_png.zip

### Generate Train / Test Lists

In [ ]:
# Create 2 files indicating which file is for training or testing

import numpy as np
import glob
import scipy.io as sio 
import os
from random import shuffle

data_folder = ['breaking98_png/', 'locking98_png/', 'popping99_png/', 'shuffle107_png/']
dataLabel = []
for i in range(3):
    data = os.listdir(data_folder[i])
    label = str(i)
    for j in data:
        dataLabel.append(data_folder[i] + j + ' '+ label)

# shuffle randomly and split into train test
shuffle(dataLabel)
num = round(0.7*len(dataLabel))
train_images = dataLabel[:num]
test_images = dataLabel[num:]

fid = open('train.list','w')
for i,p in enumerate(train_images):
    fid.write(p + '\n') 
fid.close()

fid = open('test.list','w')
for i,p in enumerate(test_images):
    fid.write(p + '\n')
fid.close()

### Build the CNN and Train

In [0]:
""" cnn_model.py """
import tensorflow as tf

def _variable_with_weight_decay(name, shape, wd):
  with tf.device('/cpu:0'):
    var = tf.get_variable(name, shape, initializer=tf.contrib.layers.xavier_initializer())
  if wd is not None:
    weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
    tf.add_to_collection('losses', weight_decay)
  return var

def inference(X,phase=False,dropout_rate=0.8,n_classes=3,weight_decay=1e-4):
    # logits should be of dimension (batch_size, n_classes)
    
    # I code in this order:
    	# Convolution -> activation -> Pooling -> Dropout -> BatchNormalization
    	# Convolution -> activation -> BatchNormalization -> Pooling -> Dropout (andrew ng)

# follow the fashion of VGG
    
# conv1
  with tf.variable_scope('conv1') as scope:
    kernel = _variable_with_weight_decay('weights',shape=[3, 3, 3, 64],wd=None)
    conv = tf.nn.conv2d(X, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_with_weight_decay('biases', [64], wd=None)
    pre_activation = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(pre_activation, name=scope.name)
  pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
  dropout1 = tf.nn.dropout(pool1, dropout_rate, noise_shape=None,seed=None, name='dropout1')
  norm1 = tf.contrib.layers.batch_norm(dropout1, center=True, scale=True, is_training=phase)
  # output size is [batchsize, 16, 16, 64]
# ?,
  
# conv2
  with tf.variable_scope('conv2') as scope:
    kernel = _variable_with_weight_decay('weights',shape=[3, 3, 64, 128],wd=None)
    conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_with_weight_decay('biases', [128], wd=None)
    pre_activation = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(pre_activation, name=scope.name)
  pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
  dropout2 = tf.nn.dropout(pool2, dropout_rate, noise_shape=None,seed=None, name='dropout2')
  norm2 = tf.contrib.layers.batch_norm(dropout2, center=True, scale=True, is_training=phase)
  # output size is [batchsize, 8, 8, 128]

# conv3
  with tf.variable_scope('conv3') as scope:
    kernel = _variable_with_weight_decay('weights',shape=[3, 3, 128, 256],wd=None)
    conv = tf.nn.conv2d(norm2, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_with_weight_decay('biases', [256], wd=None)
    pre_activation = tf.nn.bias_add(conv, biases)
    conv3 = tf.nn.relu(pre_activation, name=scope.name)
  pool3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool3')
  dropout3 = tf.nn.dropout(pool3, dropout_rate, noise_shape=None,seed=None, name='dropout3')
  norm3 = tf.contrib.layers.batch_norm(dropout3, center=True, scale=True, is_training=phase)
  # output size is [batchsize, 4, 4, 256]

# conv4
  with tf.variable_scope('conv4') as scope:
    kernel = _variable_with_weight_decay('weights',shape=[3, 3, 256, 512],wd=None)
    conv = tf.nn.conv2d(norm3, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_with_weight_decay('biases', [512], wd=None)
    pre_activation = tf.nn.bias_add(conv, biases)
    conv4 = tf.nn.relu(pre_activation, name=scope.name)
  pool4 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool4')
  dropout4 = tf.nn.dropout(pool4, dropout_rate, noise_shape=None,seed=None, name='dropout4')
  norm4 = tf.contrib.layers.batch_norm(dropout4, center=True, scale=True, is_training=phase)
  # output size is [batchsize, 2, 2, 512]

# classifier layer
  with tf.variable_scope('dense') as scope:
    norm_flat = tf.reshape(norm4, [-1, 2 * 2 * 512])
    logits = tf.layers.dense(inputs=norm_flat, units=3)

  return logits

In [0]:
import os
import numpy as np
import tensorflow as tf
import time
import random
# import cnn_model
from random import shuffle
import cv2
import scipy.io as sio

In [0]:
batch_size = 8                      # YOU MAY MODIFY THIS
max_epoch = 20                       # YOU MAY MODIFY THIS
init_lr = 1e-3                      # YOU MAY MODIFY THIS
summary_ckpt = 50                   # YOU MAY MODIFY THIS
model_ckpt = 500                    # YOU MAY MODIFY THIS
model_save_path = './model'         
tensorboard_path = './Tensorboard'
n_class = 3
image_height = 32
image_width = 32
num_channels = 3
use_pretrained_model = False

if not os.path.exists(model_save_path):
   os.mkdir(model_save_path)

In [0]:
def get_loss(logits, labels):
# FILL IN; cross entropy loss between logits and labels
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=3)
  ce_loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels,logits=logits)
#   ce_loss = tf.reduce_mean(ce_loss, name='cross_entropy')
  tf.add_to_collection('losses', ce_loss)
  losses = tf.get_collection('losses')
  total_loss = tf.add_n(losses, name='total_loss')
  return total_loss

def read_data(L, readpos=None):
  image = []
  label = []
  if readpos is None:
    readpos = random.sample(range(len(L)), batch_size)
  for i in range(len(readpos)):
# FILL IN. Read images and label. image should be of dimension (batch_size,32,32,3) and label of dimension (batch_size,)
    index = readpos[i]
    path_i = L[index][0:-1].split(' ')[0]
    label_i = L[index][0:-1].split(' ')[1]
    # [0:-1] remove '\n'
    image_i = cv2.imread(path_i)[:,:,::-1]
    # resize the image to shape of (32,32)
    image_i = cv2.resize(image_i, (32,32)) 
    #
    image.append(image_i)
    label.append(label_i)
  return np.array(image).astype('float32')/128 - 1, np.array(label).astype('int64')

In [63]:
# if run this block for multiple times
tf.reset_default_graph()

# Placeholders
learning_rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)
images = tf.placeholder(tf.float32, [None, image_height, image_width, num_channels])
labels = tf.placeholder(tf.int64, [None])
phase = tf.placeholder(tf.bool, [])
gpu_number = 0
with tf.device('/gpu:%d' %gpu_number):
  logits = inference(images, phase=phase, dropout_rate=keep_prob)

  # logits is of shape [batch_size, 10]
  # labels is of shape [batch_size, 1]
  var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
  loss = get_loss(logits, labels)

  # FILL IN. Obtain accuracy of given batch of data.
  
  predictions = tf.argmax(input=logits, axis=1)
  # predictions is of shape [batch_size, 1]
  # here use my own accuracy measure
  # in place of: accuracy = tf.contrib.metrics.accuracy(predictions,labels,weights=None,name=None)
  # how is tf.contrib.metrics.accuracy different from tf.metrics.accuracy ?
  
  accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, labels), dtype=tf.float32))


apply_gradient_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # YOU MAY MODIFY THE OPTIMIZER

# Summary list
tf.summary.scalar('Total_Loss',loss)
tf.summary.scalar('Accuracy',accuracy)
# tf.summary.image('Input', images, max_outputs=batch_size)
for var in var_list:
  tf.summary.histogram(var.op.name, var)

# Initialize
config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(tensorboard_path + '/train')
# train_writer = tf.summary.FileWriter(tensorboard_path + '/train', sess.graph)

test_writer = tf.summary.FileWriter(tensorboard_path + '/test')
sess.run(init_g)
sess.run(init_l)
saver = tf.train.Saver()

# Start from scratch or load model
if use_pretrained_model==False:
  lr = init_lr
  epoch_num = 0
else: 
  lr = np.load('learning_rate.npy')
  epoch_num = np.load('epoch_num.npy')
  saver.restore(sess,model_save_path + '/model')

trlist = list(open('train.list','r'))
testlist = list(open('test.list','r'))
test_accuracy = []
train_accuracy = []
train_loss = []

# Start training
for i in range(epoch_num, max_epoch):
# Update learning rate if required
  shuffle(trlist)
  cost = 0
  true_num = 0
  step = 0
  for pos in range(0,len(trlist),batch_size):
    
    # Load batch data
    t = time.time()
    batch_images, batch_labels = read_data(trlist, range(pos,min(pos+batch_size,len(trlist))))
    dt = time.time()-t
    # Train with batch
    t = time.time()

    _, batch_cost, batch_acc, lab, pre, summary = sess.run([apply_gradient_op, loss, accuracy, labels, predictions, merged], feed_dict={images:batch_images, labels: batch_labels, learning_rate: lr, phase: True, keep_prob: 0.8})
    print('Epoch: %d, Item : %d, Loss: %.5f, Train Accuracy: %.2f, Data Time: %.2f, Network Time: %.2f' %(i+1, pos, batch_cost, batch_acc, dt, time.time()-t))
    # test the accuracy is correct or not
    # print('acc is', sum(lab == pre)/len(lab), '\n')
    cost = cost + batch_cost
    true_num = true_num + batch_acc*len(batch_labels)
    train_writer.add_summary(summary, global_step= i*batch_size + step)
    step = step + 1
  acc = true_num / len(trlist)
  train_loss.append(cost)
  train_accuracy.append(acc)
  print('average tr_acc is', acc)
#   train_writer.add_summary(summary, global_step=i)
  
# Test, Save model
# FILL IN. Obtain test_accuracy on the entire test set and append it to variable test_accuracy. 

  t = time.time()
  te_batch_images, te_batch_labels = read_data(testlist, range(len(testlist)))
  dt = time.time()-t
  t = time.time()
  cost, te_acc, te_lab, te_pre, summary = sess.run([loss, accuracy, labels, predictions, merged], feed_dict={images:te_batch_images, labels: te_batch_labels, learning_rate: lr, phase: True, keep_prob: 0.8})
  print('Epoch: %d, Item : %d, Loss: %.5f, Test Accuracy: %.2f, Data Time: %.2f, Network Time: %.2f' %(i+1, pos, cost, te_acc, dt, time.time()-t))
  print('te_acc is', sum(te_lab == te_pre)/len(te_lab), '\n')
  test_accuracy.append(te_acc)
  test_writer.add_summary(summary, global_step=i)
  
  np.save('test_accuracy.npy',test_accuracy); sio.savemat('test_accuracy.mat', mdict={'test_accuracy': test_accuracy})
  np.save('train_accuracy.npy',train_accuracy); sio.savemat('train_accuracy.mat', mdict={'train_accuracy': train_accuracy})
  np.save('train_loss.npy',train_loss); sio.savemat('train_loss.mat', mdict={'train_loss': train_loss})
  # for restoring
  np.save('learning_rate.npy', lr)
  np.save('epoch_num.npy', i)
  saver.save(sess,model_save_path + '/model')

print('Training done.')

Epoch: 1, Item : 0, Loss: 3.21975, Train Accuracy: 0.12, Data Time: 0.00, Network Time: 0.47
Epoch: 1, Item : 8, Loss: 9.45123, Train Accuracy: 0.12, Data Time: 0.00, Network Time: 0.06
Epoch: 1, Item : 16, Loss: 1.69162, Train Accuracy: 0.62, Data Time: 0.00, Network Time: 0.07
Epoch: 1, Item : 24, Loss: 1.21739, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.07
Epoch: 1, Item : 32, Loss: 5.06338, Train Accuracy: 0.38, Data Time: 0.00, Network Time: 0.07
Epoch: 1, Item : 40, Loss: 1.04438, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.07
Epoch: 1, Item : 48, Loss: 0.80331, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.07
Epoch: 1, Item : 56, Loss: 1.44551, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.06
Epoch: 1, Item : 64, Loss: 5.58601, Train Accuracy: 0.50, Data Time: 0.00, Network Time: 0.07
Epoch: 1, Item : 72, Loss: 4.92383, Train Accuracy: 0.62, Data Time: 0.00, Network Time: 0.09
Epoch: 1, Item : 80, Loss: 1.40324, Train Accuracy: 0.88, Data

Epoch: 6, Item : 120, Loss: 0.18360, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 6, Item : 128, Loss: 0.79766, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.08
Epoch: 6, Item : 136, Loss: 0.07919, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 6, Item : 144, Loss: 0.15660, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 6, Item : 152, Loss: 0.09835, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 6, Item : 160, Loss: 0.09875, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 6, Item : 168, Loss: 0.22688, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 6, Item : 176, Loss: 1.18941, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.08
Epoch: 6, Item : 184, Loss: 0.75574, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.08
Epoch: 6, Item : 192, Loss: 0.43679, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.09
Epoch: 6, Item : 200, Loss: 0.16666, Train Accurac

Epoch: 12, Item : 24, Loss: 0.21886, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 32, Loss: 0.43752, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.07
Epoch: 12, Item : 40, Loss: 0.62538, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 48, Loss: 0.11612, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 56, Loss: 0.99958, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 64, Loss: 1.96634, Train Accuracy: 0.75, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 72, Loss: 0.00786, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 12, Item : 80, Loss: 0.06429, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 88, Loss: 0.04667, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 12, Item : 96, Loss: 0.16606, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 12, Item : 104, Loss: 0.37788, Train Accura

Epoch: 17, Item : 144, Loss: 0.04833, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 17, Item : 152, Loss: 0.01616, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 17, Item : 160, Loss: 0.06532, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 17, Item : 168, Loss: 0.07468, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 17, Item : 176, Loss: 0.20138, Train Accuracy: 0.88, Data Time: 0.00, Network Time: 0.08
Epoch: 17, Item : 184, Loss: 0.00510, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.08
Epoch: 17, Item : 192, Loss: 0.09457, Train Accuracy: 1.00, Data Time: 0.00, Network Time: 0.09
Epoch: 17, Item : 200, Loss: 0.61051, Train Accuracy: 0.83, Data Time: 0.00, Network Time: 0.08
average tr_acc is 0.9271844654407316
Epoch: 17, Item : 200, Loss: 1.82925, Test Accuracy: 0.88, Data Time: 0.01, Network Time: 0.07
te_acc is 0.875 

Epoch: 18, Item : 0, Loss: 0.55689, Train Accuracy: 0.88, Data Time: 0.00, Network

In [ ]:
ls 

In [ ]:
cd model/

In [ ]:
ls

In [ ]:
cd /content/

In [ ]:
ls Tensorboard/train/

In [ ]:
ls Tensorboard/test/

In [ ]:
!cp Tensorboard/train/events.out.tfevents.1528492205.65bff09d319e /drive/

In [ ]:
!cp Tensorboard/test/events.out.tfevents.1528492681.65bff09d319e /drive/